In [26]:
import pandas as pd

In [27]:
df = pd.read_json(path_or_buf='gg2013.json')
# df = pd.read_json(path_or_buf='gg2015.json')
df.head(3)

,id,text,timestamp_ms,user
0,290620657987887104,JLo's dress! #eredcarpet #GoldenGlobes,2013-01-14 00:45:38,"{'screen_name': 'Dozaaa_xo', 'id': 557374298}"
1,290620657887219713,What's making Sofia Vergara's boobs stay like ...,2013-01-14 00:45:38,"{'screen_name': 'theAmberShow', 'id': 14648726}"
2,290620657828524032,RT @FabSugar: Kerry Washington is EVERYTHING. ...,2013-01-14 00:45:38,"{'screen_name': 'SweetyPW', 'id': 35498686}"


In [28]:
data = df['text']
data[:5]

0               JLo's dress! #eredcarpet #GoldenGlobes
1    What's making Sofia Vergara's boobs stay like ...
2    RT @FabSugar: Kerry Washington is EVERYTHING. ...
3       Anne Hathaway has got me living. #GoldenGlobes
4    Jennifer Lopez's lace dress? Thoughts? #Golden...
Name: text, dtype: object

In [29]:
len(data)

174643

In [30]:
# sample data if necessary
sample_size = 200000
if len(df) > sample_size:
    data = data.sample(n=sample_size)
len(data)

174643

In [31]:
# hard-coded awards
awards = ['Best Motion Picture - Drama',
         'Best Motion Picture - Musical or Comedy',
         'Best Performance by an Actress in a Motion Picture - Drama',
         'Best Performance by an Actor in a Motion Picture - Drama',
         'Best Performance by an Actress in a Motion Picture - Musical or Comedy',
         'Best Performance by an Actor in a Motion Picture - Musical or Comedy',
         'Best Performance by an Actress in a Supporting Role in any Motion Picture',
         'Best Performance by an Actor in a Supporting Role in any Motion Picture',
         'Best Director - Motion Picture',
         'Best Screenplay - Motion Picture',
         'Best Motion Picture - Animated',
         'Best Motion Picture - Foreign Language',
         'Best Original Score - Motion Picture',
         'Best Original Song - Motion Picture',
         'Best Television Series - Drama',
         'Best Television Series - Musical or Comedy',
         'Best Television Limited Series or Motion Picture Made for Television',
         'Best Performance by an Actress in a Limited Series or a Motion Picture Made for Television',
         'Best Performance by an Actor in a Limited Series or a Motion Picture Made for Television',
         'Best Performance by an Actress In A Television Series - Drama',
         'Best Performance by an Actor In A Television Series - Drama',
         'Best Performance by an Actress in a Television Series - Musical or Comedy',
         'Best Performance by an Actor in a Television Series - Musical or Comedy',
         'Best Performance by an Actress in a Supporting Role in a Series, Limited Series or Motion Picture Made for Television',
         'Best Performance by an Actor in a Supporting Role in a Series, Limited Series or Motion Picture Made for Television',
         'Cecil B. deMille Award']

In [32]:
import re

In [51]:
def reduce(line):
    # remove words "performance", "picture", "in", "a", "an", "any", "made", "for", "by", and all punctuations
    pattern = r'\bperformance\b|\bpicture\b|\bin\b|\ba\b|\ban\b|\bany\b|\bmade\b|\bfor\b|\bby\b|[^\w\s]'
    return re.sub(pattern, ' ', line.lower()).split()

awards_reduced = [sorted(set(reduce(award)), key=lambda word: reduce(award).index(word)) for award in awards]

# test
for award in awards:
    ls = reduce(award)
    print(sorted(set(ls), key=lambda word: ls.index(word)))

['best', 'motion', 'drama']
['best', 'motion', 'musical', 'or', 'comedy']
['best', 'actress', 'motion', 'drama']
['best', 'actor', 'motion', 'drama']
['best', 'actress', 'motion', 'musical', 'or', 'comedy']
['best', 'actor', 'motion', 'musical', 'or', 'comedy']
['best', 'actress', 'supporting', 'role', 'motion']
['best', 'actor', 'supporting', 'role', 'motion']
['best', 'director', 'motion']
['best', 'screenplay', 'motion']
['best', 'motion', 'animated']
['best', 'motion', 'foreign', 'language']
['best', 'original', 'score', 'motion']
['best', 'original', 'song', 'motion']
['best', 'television', 'series', 'drama']
['best', 'television', 'series', 'musical', 'or', 'comedy']
['best', 'television', 'limited', 'series', 'or', 'motion']
['best', 'actress', 'limited', 'series', 'or', 'motion', 'television']
['best', 'actor', 'limited', 'series', 'or', 'motion', 'television']
['best', 'actress', 'television', 'series', 'drama']
['best', 'actor', 'television', 'series', 'drama']
['best', 'actr

In [52]:
def generate_award_num_keywords_map(awards):
    awards_reduced = []
    award_num_keywords_map = {}
    
    for i, award in enumerate([sorted(set(reduce(award)), key=lambda word: reduce(award).index(word)) for award in awards]):
        if 'or' in award:
            num_words = len(award) - 1 - award.index('or') # number of words from "or" to the end of award name
            # test
            # print(len(award)-1, award.index('or'), num_words)
            award.remove('or')
            award_num_keywords_map[i] = len(award) - 1 - num_words # do not count "best" and words from "or" to the end of award name
        else:
            award_num_keywords_map[i] = len(award) - 1 # do not count "best"
        
#         if 'best' in award:
#             award.remove('best')

        awards_reduced.append(award)
    return award_num_keywords_map, awards_reduced

In [53]:
award_num_keywords_map, awards_reduced = generate_award_num_keywords_map(awards)

sorted(award_num_keywords_map.items(), key=lambda pair: pair[0])

[(0, 2),
 (1, 2),
 (2, 3),
 (3, 3),
 (4, 3),
 (5, 3),
 (6, 4),
 (7, 4),
 (8, 2),
 (9, 2),
 (10, 2),
 (11, 3),
 (12, 3),
 (13, 3),
 (14, 3),
 (15, 3),
 (16, 3),
 (17, 3),
 (18, 3),
 (19, 4),
 (20, 4),
 (21, 4),
 (22, 4),
 (23, 5),
 (24, 5),
 (25, 3)]

In [54]:
retweets_freq_dict = {}

def remove_retweet_prefix(line):
    # find 'RT @abc: ' where abc's length is arbitrary
    pattern = re.compile(r'\bRT @([\w\'/]*)\b: ') 
   
    match = re.search(pattern, line)
    if match:
        # store corresponding retweet without 'RT @' prefix
        string = match.group()[4:]
        if string in retweets_freq_dict:
            retweets_freq_dict[string] += 1
        else:
            retweets_freq_dict[string] = 1
        
    return re.sub(pattern, ' ', line)

In [55]:
hashtag_freq_dict = {}

def remove_hashtag(line):
    pattern = re.compile(r'#([\w\'/]*)\b')
    matches = re.findall(pattern, line)
    if matches:
        # store corresponding hashtag
        for match in matches:
            if match in hashtag_freq_dict:
                hashtag_freq_dict[match] += 1
            else:
                hashtag_freq_dict[match] = 1
        
    line = re.sub(pattern, ' ', line)
    return line

In [56]:
at_freq_dict = {}

def remove_at(line):
    pattern = re.compile(r'@([\w\'/]*)\b')
    matches = re.findall(pattern, line)
    if matches:
        # store corresponding hashtag
        for match in matches:
            if match in at_freq_dict:
                at_freq_dict[match] += 1
            else:
                at_freq_dict[match] = 1
        
    line = re.sub(pattern, ' ', line)
    return line

In [57]:
def remove_url(line):
    pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+\b')
    matches = re.findall(pattern, line)
    for match in matches:
        line = re.sub(match, ' ', line)
    return line

In [58]:
def cleanse(line):
    # replace everything to ' ' except alphanumeric character, whitespace, apostrophe, hashtag
    return re.sub(r'[^\w\s\'#]', ' ', line)

# test
cleanse("Hugh Jackman is so awesome!!! #goldenglobes")

'Hugh Jackman is so awesome    #goldenglobes'

In [59]:
import spacy
# python3 -m spacy download en
nlp = spacy.load('en')

In [60]:
def identify_entities(text):
    entities = list(nlp(text).ents)
    tags = {}
    for entity in entities:
        if entity not in tags:
            tags[' '.join(t.orth_ for t in entity).strip()]=[entity.label_]
    return tags

In [69]:
# top retweets (each as an entity)
sorted(retweets_freq_dict.items(), key=lambda pair: pair[1], reverse=True)[:50]

[('goldenglobes: ', 135540),
 ('eonline: ', 113549),
 ('PerezHilton: ', 90513),
 ('TheEllenShow: ', 66729),
 ('EmWatson: ', 45667),
 ('VanityFair: ', 32168),
 ('nbcsnl: ', 29035),
 ('CNNshowbiz: ', 27278),
 ('CiudadBizarra: ', 20028),
 ('BuzzFeed: ', 19173),
 ('EW: ', 18948),
 ('nbc: ', 18310),
 ('vulture: ', 17555),
 ('piersmorgan: ', 17336),
 ('MARLONLWAYANS: ', 15039),
 ('HuffingtonPost: ', 14371),
 ('buckhollywood: ', 13901),
 ('MarilynMonroeES: ', 13203),
 ('TVGuide: ', 12554),
 ('DavidSpade: ', 12407),
 ('THR: ', 12322),
 ('MTVNews: ', 12015),
 ('PimpBillClinton: ', 11139),
 ('cinema21: ', 10856),
 ('washingtonpost: ', 10790),
 ('ninagarcia: ', 10153),
 ('Cosmopolitan: ', 9400),
 ('peopleenespanol: ', 8599),
 ('RichardCrouse: ', 8530),
 ('prodigalsam: ', 8426),
 ('peoplemag: ', 8336),
 ('kumailn: ', 8327),
 ('DannyZuker: ', 8264),
 ('DamienFahey: ', 7884),
 ('jianghomeshi: ', 7869),
 ('HuffPostWomen: ', 7830),
 ('girlsHBO: ', 7684),
 ('MHarrisPerry: ', 7601),
 ('rogergzz: ', 7536

In [70]:
# top hashtags
sorted(hashtag_freq_dict.items(), key=lambda pair: pair[1], reverse=True)[:50]

[('GoldenGlobes', 4337876),
 ('goldenglobes', 1305145),
 ('Argo', 32266),
 ('GetGlue', 30489),
 ('Homeland', 27745),
 ('redcarpet', 26868),
 ('GoldenGlobe', 22321),
 ('Goldenglobes', 22020),
 ('JodieFoster', 18830),
 ('killingit', 17759),
 ('Girls', 17507),
 ('GOLDENGLOBES', 16562),
 ('AlfombraRojaE', 15224),
 ('Skyfall', 14360),
 ('GlobosdeOro', 13852),
 ('RedCarpet', 13851),
 ('Lincoln', 13525),
 ('LesMis', 13393),
 ('GIRLS', 12822),
 ('Adele', 11519),
 ('LesMiserables', 10894),
 ('JenniferLawrence', 10119),
 ('GG2013', 10033),
 ('homeland', 10021),
 ('GoldenGlobes2013', 9842),
 ('ERedCarpet', 8910),
 ('TinaFey', 8816),
 ('eredcarpet', 8246),
 ('AmyPoehler', 8148),
 ('jodiefoster', 7922),
 ('DowntonAbbey', 7662),
 ('AnneHathaway', 7593),
 ('DjangoUnchained', 7311),
 ('girls', 7310),
 ('Oscars', 6837),
 ('Django', 6279),
 ('LoveThoseLadies', 6194),
 ('BenAffleck', 5925),
 ('GameChange', 5838),
 ('LOVEHER', 5833),
 ('Globes', 5814),
 ('BillClinton', 5672),
 ('goldenGlobes', 5617),
 ('f

In [71]:
# top @
sorted(at_freq_dict.items(), key=lambda pair: pair[1], reverse=True)[:50]

[('goldenglobes', 124271),
 ('OfficialAdele', 44704),
 ('lenadunham', 38250),
 ('GoldenGlobes', 34171),
 ('BenAffleck', 20195),
 ('RealHughJackman', 14696),
 ('SHO_Homeland', 14217),
 ('PerezHilton', 14177),
 ('girlsHBO', 13586),
 ('SofiaVergara', 12006),
 ('eonline', 11378),
 ('taylorswift13', 10619),
 ('TNTLA', 10069),
 ('JLo', 9312),
 ('PaulEpworth', 8880),
 ('GirlsHBO', 7060),
 ('msleamichele', 6621),
 ('SelenaGomez', 6515),
 ('VanessuHudgens', 6475),
 ('steph_hart', 6225),
 ('LenaDunham', 6125),
 ('RyanGosling', 5230),
 ('kerrywashington', 4965),
 ('PixarBrave', 4739),
 ('Chanel', 4467),
 ('nbc', 4440),
 ('LesMiserables', 4378),
 ('NathanFillion', 4170),
 ('KevalBaxi', 3686),
 ('Burberry', 3435),
 ('Lewis_Damian', 3429),
 ('jessicaalba', 3326),
 ('CHANEL', 3309),
 ('tomandlorenzo', 3116),
 ('iamdoncheadle', 3023),
 ('nbcsnl', 3005),
 ('piersmorgan', 2666),
 ('CiudadBizarra', 2620),
 ('CNNshowbiz', 2498),
 ('BuzzFeed', 2496),
 ('LeoDiCaprio', 2470),
 ('EvaLongoria', 2414),
 ('aziza

In [68]:
%%time
# test
num = 0
count = 0
pattern = re.compile(r'\bpresenter|\bpresent', re.IGNORECASE)
for tweet in data:
    # remove_retweet_prefix 
    line = remove_retweet_prefix(tweet)
    # remove hashtag
    line = remove_hashtag(line)
    ###
    ### consider not removing @???
    ###
    # remove @...
    line = remove_at(line)
    # remove url
    line = remove_url(line)
    # remove punctuations
    line = cleanse(line)
    
    match = re.findall(pattern, line.lower())                  
    if match:
        tags = identify_entities(line)
        count += 1
        # check the first 10 cases
        if count < 10:
            print(num)
            print(match)
            print(tweet)
            print(line)
            print(tags.keys())
            print()
    num += 1
    
print(count)

48
['present']
Definetely Kate's top designers are present in the #GoldenGlobes tonight of course thanks to the #kateefect
Definetely Kate's top designers are present in the   tonight of course thanks to the  
dict_keys(['Kate', 'tonight'])

465
['present']
@JakeCortes @Jessica_Salado comentando los #GoldenGlobes #GoldenGlobesNex mejores presentadores no hay @NexPanama
    comentando los     mejores presentadores no hay  
dict_keys([])

1176
['present']
RT @VictorBejerano: @JakeCortes @Jessica_Salado comentando los #GoldenGlobes #GoldenGlobesNex mejores presentadores no hay @NexPanama
     comentando los     mejores presentadores no hay  
dict_keys(['', 'los'])

1561
['presenter']
RT @TV3Xpose: What's your verdict on @ENews presenter Giuliana Rancic's Oriett Domenech dress? #goldenglobes http://t.co/lesP5H0F
 What's your verdict on   presenter Giuliana Rancic's Oriett Domenech dress     
dict_keys(["Giuliana Rancic 's", 'Oriett Domenech'])

1976
['presenter']
THISSSSSSSSSS RT @as_seeno

In [74]:
def find_award_presenter(award_index, verbose=False):
    pattern = re.compile(r'\bpresenter|\bpresent|\bannounce', re.IGNORECASE)
    entity_dict = {}
    
    print('Predicting for:', awards[award_index])
    
    num = 0
    for tweet in data:
        # remove_retweet_prefix 
        line = remove_retweet_prefix(tweet)
        # remove hashtag
        line = remove_hashtag(line)
        # remove @...
        line = remove_at(line)
        # remove url
        line = remove_url(line)
        # remove punctuations
        line = cleanse(line)
        
        award = awards_reduced[award_index]
        num_keywords_to_match = award_num_keywords_map[award_index]
        
        # test for 'best actress in a motion picture comedy or musical'
#         if len(set(award).intersection(set(line.split()))) > 5:
        
        if len(set(award).intersection(set(line.split()))) > num_keywords_to_match:
            
            match = re.findall(pattern, line.lower())
            if match:
                tags = identify_entities(line)

                if verbose:
                    # print the first 5 occurrences
                    if num < 5:
                        print(match)
                        print(tweet)
                        print(tags)
                        print()

                for entity in tags.keys():
                    if len(entity) > 1:
                        if entity not in entity_dict:
                            entity_dict[entity] = 1
                        else:
                            entity_dict[entity] += 1
                num += 1
            
    print('num of matches:', num)
    return entity_dict

# Predictions for Presenter

In [75]:
%%time
for key in award_num_keywords_map.keys():
    print(sorted(find_award_presenter(key, verbose=False).items(), key=lambda pair: pair[1], reverse=True)[:100])
    print()

Predicting for: Best Motion Picture - Drama
num of matches: 9
[('Lincoln', 7), ('Julia Roberts', 6), ('Lancome Ambassadress', 2), ('Roberts', 1)]

Predicting for: Best Motion Picture - Musical or Comedy
num of matches: 2
[('Jay Leno', 1), ('Jimmy Fallon', 1)]

Predicting for: Best Performance by an Actress in a Motion Picture - Drama
num of matches: 0
[]

Predicting for: Best Performance by an Actor in a Motion Picture - Drama
num of matches: 0
[]

Predicting for: Best Performance by an Actress in a Motion Picture - Musical or Comedy
num of matches: 1
[]

Predicting for: Best Performance by an Actor in a Motion Picture - Musical or Comedy
num of matches: 0
[]

Predicting for: Best Performance by an Actress in a Supporting Role in any Motion Picture
num of matches: 0
[]

Predicting for: Best Performance by an Actor in a Supporting Role in any Motion Picture
num of matches: 0
[]

Predicting for: Best Director - Motion Picture
num of matches: 0
[]

Predicting for: Best Screenplay - Motion

In [18]:
def weighted_freq(element):
    return entity_dict[element] * len(element)

e = entity_dict.copy()
for cluster in names_clusters_reduced:
    # select the longest name
    selected_name = max(cluster, key=weighted_freq)
    cluster.remove(selected_name)
    for name in cluster:
        # if not deleted in previous cases
        if name in e and selected_name in e:
            e[selected_name] += e[name]
            del e[name]